In [ ]:
# This codes are using cross-validation on the linear regreassion between vocxel matrices (V) and Units matrices (U)

# To prepare the V matrices (voxel activation):

import os
import numpy as np
from scipy.io import loadmat

# Path to the subject folder
subject_folder = r"C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\betafiles\mindy101"

# Define paths to the left and right hemisphere files
left_hem_path = os.path.join(subject_folder, "LeftHemUnsm12conds.mat")
right_hem_path = os.path.join(subject_folder, "RightHemUnsm12conds.mat")

# Load the .mat files
left_hem_data = loadmat(left_hem_path)
right_hem_data = loadmat(right_hem_path)

# Extract condition keys (ignoring metadata keys)
condition_keys = [key for key in left_hem_data.keys() if not key.startswith('__')]

# Combine all conditions into a single matrix (voxel numbers x 12)
left_hem_matrix = np.column_stack([left_hem_data[key] for key in condition_keys])
right_hem_matrix = np.column_stack([right_hem_data[key] for key in condition_keys])

# Check the shapes
print(f"Left Hemisphere Matrix Shape: {left_hem_matrix.shape}")  # Expected: (voxel numbers, 12)
print(f"Right Hemisphere Matrix Shape: {right_hem_matrix.shape}")  # Expected: (voxel numbers, 12)

# Save the left hemisphere matrix
left_output_path = os.path.join(subject_folder, "left_hem_matrix.npy")
np.save(left_output_path, left_hem_matrix)
print(f"Left hemisphere matrix saved to: {left_output_path}")

# Save the right hemisphere matrix
right_output_path = os.path.join(subject_folder, "right_hem_matrix.npy")
np.save(right_output_path, right_hem_matrix)
print(f"Right hemisphere matrix saved to: {right_output_path}")



In [ ]:
# To prepare the U matrices (units activation in DNN):
import numpy as np
import os

# Define layer names as per your DNN structure
layer_names = [ 
    'conv1', 'rnorm1', 'pool1', 'conv2', 'rnorm2', 'pool2',
    'conv3', 'conv4_W', 'conv5_W', 'pool5_W', 'conv4_G', 'conv5_G', 'pool5_G'
]

# Path to .npy files for activations
activation_path = r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili'

# Dictionary to store each layer's activation data
global activation_matrices
activation_matrices = {}

# Load and process each layer's .npy file
for layer_name in layer_names:
    # Adjust file name to match your format
    file_path = os.path.join(activation_path, f'reduced_reduced_activation_matrix_{layer_name}.npy')
    
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"File not found for layer: {layer_name}")
        continue
    
    # Load the data
    layer_data = np.load(file_path)
    
    # Check if the number of columns matches the expected size (12 conditions)
    num_rows, num_cols = layer_data.shape
    if num_cols != 12:
        raise ValueError(f"Unexpected number of columns in {layer_name}: {num_cols}, expected 12.")
    
    # Print information about the layer shape
    print(f"Layer {layer_name}: shape {layer_data.shape} (keeping as is)")
    
    # Store the data without modification
    activation_matrices[layer_name] = layer_data

# Optional: Check the shapes to confirm all layers are correctly loaded
for layer_name, data in activation_matrices.items():
    print(f"{layer_name}: shape {data.shape}")



In [30]:
# This code is for the cross-validation on the ridge regression:
# To test, using mindy101 only:

import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from itertools import product

# Load voxel matrices for the left and right hemispheres
left_hem_matrix = np.load(r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\betafiles\mindy101\left_hem_matrix.npy')
right_hem_matrix = np.load(r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\betafiles\mindy101\right_hem_matrix.npy')

# Do not standardize, use the raw voxel matrices
left_hem_matrix_standardized = left_hem_matrix  # Keep original left hemisphere matrix
right_hem_matrix_standardized = right_hem_matrix  # Keep original right hemisphere matrix

# Check the shapes
print(f"Left Hemisphere Matrix Shape (standardized): {left_hem_matrix_standardized.shape}")
print(f"Right Hemisphere Matrix Shape (standardized): {right_hem_matrix_standardized.shape}")

# Music and speech condition indices
music_conditions = list(range(6))  # Columns 0-5
speech_conditions = list(range(6, 12))  # Columns 6-11

# Loop through each layer for independent cross-validation
cv_results = {'left': {}, 'right': {}}

for layer_name, U_matrix in activation_matrices.items():
    print(f"Processing layer: {layer_name}")
    
    # Standardize the U matrix
    scaler = StandardScaler()
    U_matrix_standardized = scaler.fit_transform(U_matrix.T).T  # Standardize across features
    
    # Transpose U matrix for regression (shape: 12 x n_units)
    X = U_matrix_standardized.T  # Conditions as rows, units as columns
    
    # Left Hemisphere Cross-Validation
    print("  Left Hemisphere:")
    left_mse = []
    left_r2 = []
    for music_idx, speech_idx in product(music_conditions, speech_conditions):
        test_idx = [music_idx, speech_idx]
        train_idx = list(set(range(12)) - set(test_idx))
        
        # Train and test splits for left hemisphere
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = left_hem_matrix_standardized[train_idx], left_hem_matrix_standardized[test_idx]
        
        # Ridge regression
        ridge = Ridge(alpha=1.0)
        ridge.fit(X_train, y_train)
        y_pred = ridge.predict(X_test)
        
        # Calculate MSE
        mse = mean_squared_error(y_test, y_pred)
        left_mse.append(mse)
        
        # Calculate R^2
        r2 = r2_score(y_test, y_pred, multioutput='uniform_average')
        left_r2.append(r2)
    
    # Save results for left hemisphere
    cv_results['left'][layer_name] = {
        'mean_mse': np.mean(left_mse),
        'mean_r2': np.mean(left_r2),
    }
    print(f"    Mean MSE (Left): {np.mean(left_mse):.4f}, Mean R^2 (Left): {np.mean(left_r2):.4f}")
    
    # Right Hemisphere Cross-Validation
    print("  Right Hemisphere:")
    right_mse = []
    right_r2 = []
    for music_idx, speech_idx in product(music_conditions, speech_conditions):
        test_idx = [music_idx, speech_idx]
        train_idx = list(set(range(12)) - set(test_idx))
        
        # Train and test splits for right hemisphere
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = right_hem_matrix_standardized[train_idx], right_hem_matrix_standardized[test_idx]
        
        # Ridge regression
        ridge = Ridge(alpha=1.0)
        ridge.fit(X_train, y_train)
        y_pred = ridge.predict(X_test)
        
        # Calculate MSE
        mse = mean_squared_error(y_test, y_pred)
        right_mse.append(mse)
        
        # Calculate R^2
        r2 = r2_score(y_test, y_pred, multioutput='uniform_average')
        right_r2.append(r2)
    
    # Save results for right hemisphere
    cv_results['right'][layer_name] = {
        'mean_mse': np.mean(right_mse),
        'mean_r2': np.mean(right_r2),
    }
    print(f"    Mean MSE (Right): {np.mean(right_mse):.4f}, Mean R^2 (Right): {np.mean(right_r2):.4f}")

# Print final cross-validation results
print("\nFinal Cross-Validation Results:")
print("Left Hemisphere:", cv_results['left'])
print("Right Hemisphere:", cv_results['right'])

  


Left Hemisphere Matrix Shape (standardized): (19072, 12)
Right Hemisphere Matrix Shape (standardized): (17995, 12)
Processing layer: conv1
  Left Hemisphere:
    Mean MSE (Left): 0.7488, Mean R^2 (Left): -5769.6060
  Right Hemisphere:
    Mean MSE (Right): 0.7234, Mean R^2 (Right): -724.3379
Processing layer: rnorm1
  Left Hemisphere:
    Mean MSE (Left): 0.7392, Mean R^2 (Left): -9817.8339
  Right Hemisphere:
    Mean MSE (Right): 0.7191, Mean R^2 (Right): -517.3307
Processing layer: pool1
  Left Hemisphere:
    Mean MSE (Left): 0.9046, Mean R^2 (Left): -9302.7970
  Right Hemisphere:
    Mean MSE (Right): 0.6791, Mean R^2 (Right): -527.5690
Processing layer: conv2
  Left Hemisphere:
    Mean MSE (Left): 0.7411, Mean R^2 (Left): -6038.0577
  Right Hemisphere:
    Mean MSE (Right): 0.7040, Mean R^2 (Right): -693.4882
Processing layer: rnorm2
  Left Hemisphere:
    Mean MSE (Left): 0.7653, Mean R^2 (Left): -4396.2558
  Right Hemisphere:
    Mean MSE (Right): 0.7519, Mean R^2 (Right): -79

In [29]:
# only ridge regression without cross-validation
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Load voxel matrices for the left and right hemispheres
left_hem_matrix = np.load(r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\betafiles\mindy101\left_hem_matrix.npy')
right_hem_matrix = np.load(r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\betafiles\mindy101\right_hem_matrix.npy')

# Standardize voxel matrices (V) for left and right hemispheres
scaler_left = StandardScaler()
scaler_right = StandardScaler()

left_hem_matrix_standardized = scaler_left.fit_transform(left_hem_matrix)
right_hem_matrix_standardized = scaler_right.fit_transform(right_hem_matrix)

# Loop through each layer and perform Ridge regression
results = {'left': {}, 'right': {}}

for layer_name, U_matrix in activation_matrices.items():
    print(f"Processing layer: {layer_name}")
    
    # Standardize the U matrix
    scaler = StandardScaler()
    U_matrix_standardized = scaler.fit_transform(U_matrix.T).T  # Standardize across features
    
    # Transpose U matrix for regression (shape: 12 x n_units)
    X = U_matrix_standardized.T  # Conditions as rows, units as columns
    
    # Ridge regression for Left Hemisphere
    left_mse_list = []
    left_r2_list = []
    for voxel_idx in range(left_hem_matrix_standardized.shape[0]):  # Loop over rows (voxels)
        y = left_hem_matrix_standardized[voxel_idx, :]  # Correctly extract a row for a single voxel
        
        ridge = Ridge(alpha=1.0)
        ridge.fit(X, y)
        y_pred = ridge.predict(X)
        
        # Calculate MSE and R^2
        mse = mean_squared_error(y, y_pred)
        r2 = r2_score(y, y_pred)
        
        left_mse_list.append(mse)
        left_r2_list.append(r2)
    
    # Save results for left hemisphere
    results['left'][layer_name] = {
        'mean_mse': np.mean(left_mse_list),
        'mean_r2': np.mean(left_r2_list),
    }
    print(f"    Left Hemisphere: Mean MSE={np.mean(left_mse_list):.4f}, Mean R^2={np.mean(left_r2_list):.4f}")
    
    # Ridge regression for Right Hemisphere
    right_mse_list = []
    right_r2_list = []
    for voxel_idx in range(right_hem_matrix_standardized.shape[0]):  # Loop over rows (voxels)
        y = right_hem_matrix_standardized[voxel_idx, :]  # Correctly extract a row for a single voxel
        
        ridge = Ridge(alpha=1.0)
        ridge.fit(X, y)
        y_pred = ridge.predict(X)
        
        # Calculate MSE and R^2
        mse = mean_squared_error(y, y_pred)
        r2 = r2_score(y, y_pred)
        
        right_mse_list.append(mse)
        right_r2_list.append(r2)
    
    # Save results for right hemisphere
    results['right'][layer_name] = {
        'mean_mse': np.mean(right_mse_list),
        'mean_r2': np.mean(right_r2_list),
    }
    print(f"    Right Hemisphere: Mean MSE={np.mean(right_mse_list):.4f}, Mean R^2={np.mean(right_r2_list):.4f}")

# Print final results
print("\nFinal Results Without Cross-Validation:")
print("Left Hemisphere:", results['left'])
print("Right Hemisphere:", results['right'])





Processing layer: conv1
    Left Hemisphere: Mean MSE=0.0002, Mean R^2=0.9996
    Right Hemisphere: Mean MSE=0.0002, Mean R^2=0.9997
Processing layer: rnorm1
    Left Hemisphere: Mean MSE=0.0002, Mean R^2=0.9997
    Right Hemisphere: Mean MSE=0.0002, Mean R^2=0.9997
Processing layer: pool1
    Left Hemisphere: Mean MSE=0.0002, Mean R^2=0.9997
    Right Hemisphere: Mean MSE=0.0002, Mean R^2=0.9997
Processing layer: conv2
    Left Hemisphere: Mean MSE=0.0002, Mean R^2=0.9996
    Right Hemisphere: Mean MSE=0.0002, Mean R^2=0.9997
Processing layer: rnorm2
    Left Hemisphere: Mean MSE=0.0003, Mean R^2=0.9996
    Right Hemisphere: Mean MSE=0.0002, Mean R^2=0.9996
Processing layer: pool2
    Left Hemisphere: Mean MSE=0.0003, Mean R^2=0.9995
    Right Hemisphere: Mean MSE=0.0003, Mean R^2=0.9996
Processing layer: conv3
    Left Hemisphere: Mean MSE=0.0003, Mean R^2=0.9996
    Right Hemisphere: Mean MSE=0.0002, Mean R^2=0.9997
Processing layer: conv4_W
    Left Hemisphere: Mean MSE=0.0002, Mea

In [28]:
print(y.shape)

(19072,)
